# Data Mining Versuch Recommender Systeme

* **Autor**: Prof. Dr. Johannes Maucher

* **Dozenten**: Manuel Eberhardinger, Florian Rupp

* **Studierende**: Paul Michels (pm080), Erzan Gashi (eg053), Patryk Gadziomski (pg058)

## Abgabe:

- **Abzugeben ist das Jupyter Notebook mit dem verlangten Implementierungen und den entsprechenden Ausgaben.**
- **Das Notebook ist als .ipynb und als .html abzugeben.**
- **Klausurelevante Fragen sind Dokument "Fragenkatalog Datamining" zu finden.**
- Antworten auf Fragen im Notebook, Diskussionen und Beschreibung der Ergebnisse sind optional (aber empfohlen) und werden nicht bewertet.

* [Übersicht Data Mining Praktikum](https://maucher.pages.mi.hdm-stuttgart.de/ai/page/dm/)


# Einführung

## Lernziele:

In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __Ähnlichkeit:__ Verfahren zur Bestimmung der Ähnlichkeit zwischen Personen (Kunden) und Elementen (Produkten)
* __Empfehlungssysteme__ Collaborative Filtering 
* __Collaborative Filtering:__ Nutzerbezogener Ansatz und elementbasierter Ansatz

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung

### Recommender Systeme

Recommender Systeme werden im E-Commerce eingesetzt um Werbung in Form von kundenspezifischen Empfehlungen zu verteilen. Weitläufig bekannt sind die Amazon-Empfehlungen, die entweder per e-mail geschickt oder nach dem Log-In in der Web-Page angezeigt werden. Diese Empfehlungen werden in Abhängigkeit von den bisher vom jeweiligen Kunden gekauften bzw. bewerteten Produkten erstellt. In diesem Versuch werden die derzeit wohl am weitest verbreiteteten Verfahren für die Erzeugung kundenspezifischer Empfehlungen vorgestellt, darunter das elementweise Collaborative Filtering, welches z.B. auch von Amazon eingesetzt wird.     

Direkt-Marketing Methoden wie die kundenspezifische Erzeugung und Bereitstellung von Werbung erfordern detaillierte Kunden- und Warenkorbanalysen. Kunden mit ähnlichem Kaufverhalten werden in Kundengruppen zusammengefasst. Die Warenkorbanalyse untersucht u.a. welche Waren bevorzugt im Verbund von der gleichen Person gekauft werden. Damit kann ein Händler Werbung in Form von Empfehlungen individuell und gezielt an seine Kunden richten, abhängig davon welcher Kundengruppe er angehört und welche Produkte bevorzugt von dieser Kundengruppe nachgefragt werden. 

Im ersten Teil der Übung werden fiktive Daten in einer überschaubaren Menge verwendet. Es handelt sich hier um Filmbewertungen. Anhand dieses Beispiels sollen die notwendigen Methoden und Abläufe implementiert und getestet werden. Diese werden im zweiten Teil der Übung auf echte Daten angewandt. Hierzu werden über eine Python-API Daten vom Internet-Meta-Radio _last.fm_ integriert. Auf der Basis dieser Daten sollen dann Musikempfehlungen für last.fm User berechnet werden. 

Recommender Systeme lassen sich mit

* Clustering Verfahren
* Suchalgorithmen
* Collaborativen Filtering 
 
realisieren. Am häufigsten wird hierbei das Collaborative Filtering eingesetzt. Für das Collaborative Filtering wird jeder der $M$ User durch einen $N$-dimensionalen Vektor beschrieben, wobei $N$ die Anzahl der Produkte im Angebot des Händlers ist. Jedes Element im Vektor gehört zu einem speziellen Produkt. Das Element hat den Wert 1, wenn der User dieses Produkt bereits gekauft hat, sonst 0 (andere Wertbelegungen sind möglich, z.B. wenn Produktbewertungen vorliegen). Alle $M$ Zeilenvektoren können zur _User/Item_ Matrix zusammengefasst werden (siehe Abbildung).


<img src="https://maucher.home.hdm-stuttgart.de/Pics/UserItemMatrix.png" style="width:1000px" align="center">

Das traditionelle **userbasierte Collaborative Filtering (UCF)**, benutzt die Ähnlichkeit zwischen Benutzern: Um für User $U_i$ eine Empfehlung zu erzeugen wird zunächst der diesem User ähnlichste Kunde (oder eine Menge vom ähnlichsten Kunden) ermittelt. Dann werden $U_i$ die Produkte (Items) empfohlen, welche der ähnlichste Kunde gekauft hat, $U_i$ selbst jedoch noch nicht. 

Dieser Ansatz skaliert schlecht im Fall sehr großer *User/Item*-Matrizen. Ausserdem ist er für User, welche erst wenige Produkte gekauft haben unzuverlässig. Besser eignet sich in diesen Fällen das **itembasierte Collaborative Filtering (ICF)**. Es wird u.a. von Amazon.com eingesetzt. Diese Variante benutzt die Ähnlichkeit zwischen Produkten (Items). Dabei sind Produkte umso ähnlicher je mehr Kunden diese Produkte gemeinsam gekauft haben. Für die Produkte welche ein Referenzuser $U_i$ bereits gekauft hat, werden die ähnlichsten Produkte ermittelt. Diese ähnlichsten Produkte werden $U_i$ empfohlen, wenn er sie nicht schon selbst gekauft hat.

Im folgenden Abschnitt werden einige gebräuchliche Metriken für die Berechnung der Ähnlichkeit zwischen Benutzern oder Artikeln vorgestellt. Für Collaboratives Filtering wird sehr häufig das Cosinus - Ähnlichkeitsmaß eingesetzt.


### Gebräuchliche Ähnlichkeitsmaße

Die __euklidische Distanz__ $d_E(\underline{a},\underline{b})$ zwischen zwei n-dimensionalen Vektoren $\underline{a}=(a_1,\ldots,a_n)$ und $\underline{b}=(b_1,\ldots,b_n)$ berechnet sich zu
	$$
	d_E(\underline{a},\underline{b})=\sqrt{\sum_{i=1}^n (a_i-b_i)^2}
	$$
Zwei Vektoren können als umso ähnlicher erachtet werden, je kleiner deren euklidische Distanz ist. 
Ein auf der euklidischen Metrik basierendes Ähnlichkeitsmaß zwischen zwei Vektoren $\underline{a}$ und $\underline{b}$ kann durch 
$$
s_E(\underline{a},\underline{b})=\frac{1}{1+d_E(\underline{a},\underline{b})}
$$
angegeben werden.


__Pearson Korrelation__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Pearson-Korrelationskoeffizient $\rho_{\underline{a},\underline{b}}$ ausgedrückt werden. Er berechnet sich zu
$$
\rho_{\underline{a},\underline{b}}= \frac{1}{N}\cdot \sum\limits_{i=1}^{N}\frac{(a_i-\overline{a})}{\sigma_a} \frac{(b_i-\overline{b})}{\sigma_b}
$$
Dabei bezeichnet $N$ die Länge der Vektoren, $\overline{a}$ den Mittelwert und $\sigma_a$ die Standardabweichung des Vektors $\underline{a}$. 

Der Pearson-Korrelationskoeffizient misst die lineare Abhängigkeit zwischen zwei Vektoren. Der maximale Wert von $+1$ wird erreicht, wenn die durch die beiden Vektoren definierten N Punkte im 2-dimensionalen Raum auf einer ansteigenden Geraden liegen. Der Minimalwert von $-1$ wird erreicht, wenn die Punkte auf einer abfallenden Geraden liegen. Der Betrag des Koeffizienten ist umso kleiner, je stärker die Punkte von einer fiktiven Geraden (kann durch lineare Regression berechnet werden) abweichen. Der Koeffizient ist $0$ wenn keine lineare Abhängigkeit zwischen den Vektoren besteht.


__Cosinus Ähnlichkeitsmaß__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Cosinus $\cos(\underline{a},\underline{b})$ ausgedrückt werden. Er berechnet sich zu
$$
\cos(\underline{a},\underline{b})= \frac{\underline{a} \cdot \underline{b}}{\left\|\underline{a}\right\|\cdot \left\|\underline{b}\right\|}
$$
wobei im Zähler das Skalarprodukt der beiden Vektoren steht und mit $\left\|\underline{x}\right\|$ der Betrag des Vektors $\underline{x}$ bezeichnet wird.

Falls die Vektoren $\underline{a}$ und $\underline{b}$ mittelwertfrei sind, ist der Cosinus-Ähnlichkeitswert gleich dem Pearson-Korrelationswert. In der Dokument- und Textanalyse wird vornehmlich das Cosinus-Ähnlichkeitsmaß verwendet. 


__Russel Rao Ähnlichkeitsmaß__
Die Russel Rao-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert 1 haben und der Länge $n$ der Vektoren. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Russel-Rao-Ähnlichkeit $s_{RR}(\underline{a},\underline{b})=2/6=0.333$.

__Jaccard Ähnlichkeitsmaß__
Die Jaccard-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert $1$ haben und der Anzahl der Stellen in denen mindestens einer der beiden Vektoren ungleich $0$ ist. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Jaccard-Ähnlichkeit $s_{J}(\underline{a},\underline{b})=2/5=0.4$. Die Jaccard Metrik wird in diesem Versuch für die Bestimmung der Ähnlichkeit von *last.fm*-Usern eingesetzt.


## Vor dem Versuch zu klärende Fragen
Eine Untermenge der im Folgenden aufgeführten Fragen wird zu Beginn des Versuchs im Rahmen eines Gruppenkolloqs abgefragt. Auf jede Frage sollte von mindestens einem Gruppenmitglied eine Antwort geliefert werden und jedes Gruppenmitglied muss mindestens eine der gestellten Fragen beantworten können.

**Aufgaben:**

* Beschreiben Sie das Prinzip des userbasierten Collaborativen Filtering (UCF).

**Antwort**:

Das **userbasierte Collaborative Filtering (UCF)**, benutzt die Ähnlichkeit zwischen Benutzern: Um für Benutzer $U_i$ eine Empfehlung zu erzeugen, wird zunächst der diesem Benutzer ähnlichste Kunde (oder eine Menge vom ähnlichsten Kunden) ermittelt. Dann werden $U_i$ die Produkte empfohlen, welche der ähnlichste Kunde gekauft hat, $U_i$ selbst jedoch noch nicht.

* Welche Nachteile hat das UCF?

**Antwort**:

* Dieser Ansatz skaliert schlecht im Fall sehr großer *Benutzer/Artikel*-Matrizen
* Ausserdem ist er für Benutzer, welche erst wenige Produkte gekauft haben unzuverlässig

* Worin besteht der Unterschied zwischen UCF und itembasierten Collaborativen Filtering (ICF)?

**Antwort**:

* Im Gegensatz zum UCF, werden beim ICF die Ähnlichkeiten bezüglich der Artikel ermittelt und nicht die der Benutzer.
(ICF: Also wenn Produkte 2, 4, 6 oft gemeinsam von Benutzern erworben werden, werden Benutzern die bereits Produkte 2 und 4 erworben haben, Produkt 6 vorgeschlagen.)

Gegeben seien die Vektoren 

$a=[1,2,3,4,5,6]$
    
$b=[3,3,5,6,7,8]$

Schreiben Sie eine Python Funktion, die den Mittelwert derartiger Vektoren berechnet. Schreiben Sie eine weitere Funktion, die die Varianz berechnet

In [1]:
import numpy as np
from pprint import pprint
import pylast

In [2]:
a = np.array([1, 2, 3, 4, 5, 6])
b = np.array([3, 3, 5, 6, 7, 8])

In [3]:
def calc_mean(vector):
    return np.sum(vector)/len(vector)

In [4]:
def calc_var(vector):
    return np.sum([(i - calc_mean(vector))**2 for i in vector])/len(vector)

In [5]:
print(f'Mittelwert a: {calc_mean(a)}')
print(f'Mittelwert b: {calc_mean(b)}')
print(f'Varianz a: {calc_var(a)}')
print(f'Varianz b: {calc_var(b)}')

Mittelwert a: 3.5
Mittelwert b: 5.333333333333333
Varianz a: 2.9166666666666665
Varianz b: 3.5555555555555554


* Wie groß ist die

    - Euklidische Ähnlichkeit
    - Pearson Ähnlichkeit
    - Cosinus Ähnlichkeit
    
    zwischen den Vektoren $\underline{a}$ und $\underline{b}$? 

In [6]:
def calc_euclidean_similarity(a, b):
    if len(a) != len(b):
        print('Die Vektoren haben nicht die gleiche Länge!')
    else:
        similarity = 1 / (1 + np.sqrt(np.sum((a - b)**2)))
    return similarity

In [7]:
def calc_pearson_similarity(a, b):
    a1 = (a - calc_mean(a)) / np.sqrt(calc_var(a))
    b1 = (b - calc_mean(b)) / np.sqrt(calc_var(b))
    return np.sum(a1 * b1) /len(a)

In [8]:
def calc_cosinus_similarity(a, b):
    numerator = (a @ b)
    denominator = np.sqrt(np.sum(a**2)) * np.sqrt(np.sum(b**2))
    return numerator / denominator

In [9]:
calc_euclidean_similarity(a, b)

np.float64(0.179128784747792)

In [10]:
calc_pearson_similarity(a, b)

np.float64(0.9833434220628549)

In [11]:
calc_cosinus_similarity(a, b)

np.float64(0.991060084745164)

* In welchen Fällen sind Cosinus- und Pearsonähnlichkeit der euklidischen Ähnlichkeit vorzuziehen?

**Antwort**:

* Die Euklidische Distanz berechnet Abstand der Vektoren, während Pearson und Cosinus die Richtung bzw. den Winkel zwischen den beiden Vektoren misst. Wenn ein großer Unterschied bei der Länge der Vektoen besteht (``Vector_a: (1, 2, 3, 5, 6)`` und ``Vector_b: (100, 200, 314, 500, 600)``) dann wären die Vektoren nach der Euklidischen Ähnlichkeit nicht ähnlich, nach der Pearson und Cosinus Ähnlichkit aber schon.

# Versuchsdurchführung
## Teil 1: Fiktive Filmbewertung
### Daten
Folgende Tabelle enthält die Filmbewertungen von 7 Personen.
from IPython.display import Latex
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:


<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmRecommendations.PNG" style="width:1000px" align="center">



Die Tabelle ist als Python dictionary _critics_ implementiert. Die Keys des Python-Dictionary definieren die Namen von Personen (Zeilen in der Matrix), die Filme bewertet haben. Die Values sind selbst wieder Dictionarys, welche als Keys die Filmnamen (Spalten in der Matrix) und als Values die jeweilige Filmbewertung (Matrixelment) enthalten.

In [12]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}
}

### Ähnlichkeiten berechnen

Für die Bestimmung der Ähnlichkeit zwischen Personen und Produkten werden in diesem Versuch ein auf der euklidischen Distanz basierendes Ähnlichkeitsmaß und die Pearson-Korrelation verwendet. Beide Ähnlichkeitsmaße sind in den unten definierten Funktionen implementiert. Alle drei hier implementierten Funktionen zur Berechnung der Ähnlichkeit erhalten als Übergabeparameter das oben definierte Dictionary, das die Filmbewertungen enthält und die Namen der zwei Personen, die verglichen werden sollen. 

Zu beachten ist, dass in beiden Funktionen für die Berechnung der Ähnlichkeit zwischen zwei Personen nur die Produkte berücksichtigt werden, welche von beiden Personen schon bewertet wurden. Es handelt sich hier also um modifizierte Ähnlichkeitsfunktionen. 

__Aufgabe:__
Fragen Sie von diesem Dictionary _Toby's_ Bewertung des Films _Snakes on a Plane_ ab und geben Sie diesen Wert aus: 

In [13]:
critics['Toby']['Snakes on a Plane']

4.5

In [14]:
def sim_euclid(prefs, person1, person2, normed=True):
    ''' Returns a euclidean-distance-based similarity score for 
    person1 and person2. In the distance calculation the sum is computed 
    only over those items, which are nonzero for both instances, i.e. only
    films which are ranked by both persons are regarded.
    If the parameter normed is True, then the euclidean distance is divided by
    the number of non-zero elements integrated in the distance calculation. Thus
    the effect of larger distances in the case of an increasing number of commonly ranked
    items is avoided.
    '''
    # Get the list of shared_items
    si={}
    for item in prefs[person1]: 
        if item in prefs[person2]: 
            si[item] = 1
            
    # len(si) counts the number of common ratings
    # if they have no ratings in common, return 0
    if len(si) == 0: 
        return 0

    # Add up the squares of all the differences
    sum_of_squares=np.sqrt(sum([pow(prefs[person1][item]-prefs[person2][item],2) 
                     for item in prefs[person1] if item in prefs[person2]]))
    if normed:
        sum_of_squares= 1.0/len(si)*sum_of_squares
    
    return 1/(1+sum_of_squares)


def sim_pearson(prefs,p1,p2):
    '''
    Returns the Pearson correlation coefficient for p1 and p2
    '''

    # Get the list of commonly rated items
    si={}
    for item in prefs[p1]: 
        if item in prefs[p2]: 
            si[item]=1

    # if they are no ratings in common, return 0
    if len(si)==0: return 0

    # Sum calculations
    n=len(si)

    # Calculate means of person 1 and 2
    mp1=np.mean([prefs[p1][it] for it in si])
    mp2=np.mean([prefs[p2][it] for it in si])

    # Calculate standard deviation of person 1 and 2
    sp1=np.std([prefs[p1][it] for it in si])
    sp2=np.std([prefs[p2][it] for it in si])

    # If all elements in one sample are identical, the standard deviation is 0. 
    # In this case there is no linear correlation between the samples
    if sp1==0 or sp2==0:
        return 0
    r=1/(n*sp1*sp2)*sum([(prefs[p1][it]-mp1)*(prefs[p2][it]-mp2) for it in si])
    return r


def sim_RusselRao(prefs,person1,person2,normed=True):
    ''' Returns RusselRao similaritiy between 2 users. The RusselRao similarity just counts the number
    of common non-zero components of the two vectors and divides this number by N, where N is the length
    of the vectors. If normed=False, the division by N is omitted.
    '''
    # Get the list of shared_items
    si={}
    commons=0
    for item in prefs[person1]: 
        if prefs[person1][item]==1 and prefs[person2][item]==1:   
            commons+=1
    #print commons
    if not normed:
        return commons
    else:
        return commons*1.0/len(prefs[person1]) 

**Aufgabe:**
1. Geben Sie die euklidische Ähnlichkeit und die Pearson Ähnlichkeit zwischen den Personen _Toby_ und _Lisa Rose_ aus.
2. Diskutieren Sie die unterschiedlichen Ähnlichkeitswert.

In [15]:
print(f"Toby und Lisa Rose: {sim_euclid(critics, 'Toby', 'Lisa Rose')}")
print(f"Toby und Lisa Rose: {sim_pearson(critics, 'Toby', 'Lisa Rose')}")

Toby und Lisa Rose: 0.615911621788925
Toby und Lisa Rose: 0.9912407071619302


**Antwort**:

* Die Vektorenlänge von Toby und Lisa Rose sind unterschiedlich lang
* Daraus ergibt sich eine geringe Ähnlichkeit nach der Euklidischen Ähnlichkeit
* Die Euklidische Ähnlichkeit ist fehleranfälliger
* Bei der Pearson Ähnlichkeit, welche die lineare Abhängigkeit beachtet, ergibt sich eine hohe Ähnlichkeit
* Der Pearson Ähnlichkeitwert ist zuverlässiger

__Aufgabe:__
0. Schreiben Sie eine Funktion `topMatches(prefs,person,similarity)`, welche für eine beliebige in _critics_ enthaltene Person die Ähnlichkeitswerte zu allen anderen Personen berechnet und in einer geordneten Liste zurück gibt. Der Funktion soll als Übergabeparameter auch die anzuwendende Ähnlichkeitsfunktion (`sim_euclid` oder `sim_pearson`) übergeben werden können. Berechnen Sie mit dieser Funktion für jede Person die *top matches*, zunächst unter Verwendung der euklidischen- dann unter Verwendung der Pearson-Ähnlichkeit.
1. Geben Sie mit der implementierten Funktion die *top matches* der Person Toby aus.
2. Vergleichen Sie die beiden Ähnlichkeitsmaße. Welches Ähnlichkeitsmaß erscheint Ihnen für diesen Anwendungsfall sinnvoller und warum?

In [ ]:
def top_matches(prefs, person, similarity, norm=True):
    sim_list = []
    sim_list_sort = []
    
    if (similarity == "sim_euclid"):
        for i in prefs:
            if (i != person):
                sim_list.append([i, sim_euclid(prefs, person, i, norm)])
        sim_list_sort = sorted(sim_list, key=lambda element: element[1], reverse = True)
        
    elif (similarity == "sim_pearson"):
        for i in prefs:
            if (i != person):
                sim_list.append([i, sim_pearson(prefs, person, i)])
        sim_list_sort = sorted(sim_list, key=lambda element: element[1], reverse = True)
        
    elif (similarity == "sim_RusselRao"):
        for i in prefs:
            if (i != person):
                sim_list.append([i, sim_RusselRao(prefs, person, i)])
        sim_list_sort = sorted(sim_list, key=lambda element: element[1], reverse = True)
        
    return sim_list_sort

In [17]:
print('Euclidic Similarities:')
top_matches(critics, 'Toby', 'sim_euclid')

Euclidic Similarities:


[['Mick LaSalle', np.float64(0.6666666666666666)],
 ['Claudia Puig', np.float64(0.6246387977050463)],
 ['Lisa Rose', np.float64(0.615911621788925)],
 ['Michael Phillips', np.float64(0.5584815598877471)],
 ['Jack Matthews', np.float64(0.5227744249483389)],
 ['Gene Seymour', np.float64(0.5108747069239427)]]

In [18]:
print('Pearson Similarities:')
top_matches(critics, 'Toby', 'sim_pearson')

Pearson Similarities:


[['Lisa Rose', np.float64(0.9912407071619302)],
 ['Mick LaSalle', np.float64(0.9244734516419051)],
 ['Claudia Puig', np.float64(0.8934051474415642)],
 ['Jack Matthews', np.float64(0.6628489803598703)],
 ['Gene Seymour', np.float64(0.3812464258315117)],
 ['Michael Phillips', np.float64(-1.0)]]

**Antwort**:

* Bei der Euklidischen Ähnlichkeit, kann sich der Wert ändern je nachdem ob die Euklidische Ähnlichkeit normiert ist oder nicht
* Die Pearson Ähnlichkeit gibt eine tatsächliche Rangliste an Empfehlungen aus
* Die Pearson Ähnlichkeit gibt auch negative Korrelationen zurück, womit man weiß, welche man ignorieren sollte

### Berechnung von Empfehlungen mit User basiertem Collaborative Filtering
Für die Produkte, die von einer Person noch nicht gekauft wurden, sollen Empfehlungen berechnet werden. Die Empfehlungen können ebenfalls Werte zwischen 1 (wird nicht empfohlen) und 5 (wird stark empfohlen) annehmen. Für die Berechnung der Empfehlung werden die Bewertungen des jeweiligen Produkts durch die anderen Personen herangezogen. Dabei werden die Bewertungen der ähnlichen Personen (d.h. hoher Pearson-Korrelationswert) stärker mit einbezogen als die Bewertungen durch Personen mit einem niedrigen Korrelationswert.

__Beispiel:__
Toby hat die Filme *The Night Listener*, _Lady in the Water_ und _Just My Luck_ noch nicht gekauft. Für diese Filme soll für Toby eine Empfehlung berechnet werden.
In der unten aufgeführten Tabelle enthält die zweite Spalte die _Pearson-Ähnlichkeitswerte_ zwischen Toby und den anderen Personen. Die Spalten 3, 5 und 7 enthalten die Bewertungen der Filme *The Night Listener*, _Lady in the Water_ und _Just My Luck_ durch die anderen Personen. Die Spalten 4, 6 und 8 enthalten die jeweilige Filmbewertung gewichtet (mulipliziert) mit den Ähnlichkeitswerten der jeweiligen Person. Es fällt auf, dass in der Tabelle _Michael_ nicht enthalten ist. Das liegt daran, dass _Michael_ und _Toby_ einen negativen Ähnlichkeitswert aufweisen, d.h. deren Interessen sind gegenläufig. Personen mit negativem Ähnlichkeitswert sollten für Empfehlungen nicht berücksichtigt werden.
Die Zeile _Sum_ enthält die Summe aller gewichteten Bewertungen. Aus diesem Wert allein kann die Empfehlung noch nicht abgeleitet werden, da Filme die nur von wenigen Personen bewertet wurden, eine relativ kleine Summe ergeben. Deshalb sollte _Sum_ noch durch die Anzahl der Bewertungen für diesen Film geteilt werden. Oder besser: Nicht durch die Summe der Bewertungen, sondern durch die Summe der relevanten Ähnlichkeitswerte (*KSum*). Der resultierende Empfehlungswert ist in der letzten Zeile eingetragen.



<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmCalculation.PNG" style="width:1000px" align="center">




__Aufgabe:__
Schreiben Sie eine Funktion *getRecommendations(prefs,person,similarity)*, mit der die Empfehlungswerte berechnet werden können und bestimmen Sie die Empfehlungswerte für Toby. Der Funktion wird  

* das Dictionary _critics_ mit den Filmbewertungen, 
* der Name der Person, für welche Empfehlungen berechnet werden sollen
* die Methode für die Berechnung der Ähnlichkeit *sim_euclid* oder *sim_pearson*

übergeben. Die Methode soll eine geordnete Liste zurück geben. Jedes Listenelement enthält an erster Stelle den berechneten Empfehlungswert und an zweiter Stelle den Namen des Films. Die Liste soll nach Empfehlungswerten absteigend geordnet sein.

Testen Sie diese Funktion indem Sie die Empfehlungen für _Toby_ berechnen und mit den Werten in der oben aufgeführten Tabelle vergleichen.

In [ ]:
def getRecommendations(prefs, person, similarity, norm=True):
    # get all Items
    all_films = []
    all_persons = []
    
    for i in prefs.keys():
        all_persons.append(i)

    # get all Items bought by person
    for i in range(0, len(all_persons)):
        films_of_user_i = prefs[all_persons[i]]
        for j in films_of_user_i:
            if(j not in all_films):
                all_films.append(j)

    # get all Items not bought by person
    films_person_has_not_seen = [] 
    for i in all_films:
        if(i not in prefs[person] or prefs[person][i] == 0):
            films_person_has_not_seen.append(i)

    # get the similarity scores between person and the other users, 
    # and filter out the pnes that dont have a positive sim_score
    sim_list_of_per = top_matches(prefs, person, similarity, norm)
    persons_of_intrest = []
    for i in range(0, len(sim_list_of_per)):
        if (sim_list_of_per[i][1] > 0):
            persons_of_intrest.append([sim_list_of_per[i][0],sim_list_of_per[i][1]])

    # calculate the recommendationscores for the items the person has not bought
    collective_weighted_rating = []
    k_sum = []

    for film in films_person_has_not_seen:
        sum_rating = 0
        sim_sum = 0
        for pers in persons_of_intrest:
            if(film in prefs[pers[0]]):
                sum_rating = sum_rating + (prefs[pers[0]][film] * pers[1])
                sim_sum = sim_sum + pers[1]
        collective_weighted_rating.append([film, sum_rating])
        k_sum.append([film, sim_sum])

    recommendation_value = []
    for i in range(0,len(k_sum)):
        if(k_sum[i][1] != 0):
                recommendation_value.append([collective_weighted_rating[i][1]/k_sum[i][1], k_sum[i][0]])

    recommendation_value_sorted = sorted(recommendation_value, key=lambda element: element[0], reverse=True)
    return(recommendation_value_sorted)

In [20]:
getRecommendations(critics, 'Toby', 'sim_euclid')

[[np.float64(3.4273481378103883), 'The Night Listener'],
 [np.float64(2.795737031164006), 'Lady in the Water'],
 [np.float64(2.407392750287351), 'Just My Luck']]

In [21]:
getRecommendations(critics, 'Toby', 'sim_pearson')

[[np.float64(3.3477895267131013), 'The Night Listener'],
 [np.float64(2.8325499182641622), 'Lady in the Water'],
 [np.float64(2.5309807037655645), 'Just My Luck']]

### Berechnung von Empfehlungen mit Item basiertem Collaborative Filtering
In den vorigen Aufgaben wurden Ähnlichkeiten zwischen Personen bestimmt und für Produktempfehlungen benutzt (User basiertes Collaborative Filtering). Jetzt soll die Ähnlichkeit zwischen Produkten berechnet werden und auf der Basis dieser Produktähnlichkeit Empfehlungen berechnet werden (Item basiertes Collaborative Filtering).

Dabei sollen die bereits implementierten Ähnlichkeitsfunktion *sim_euclid* und *sim_pearson* sowie die Ähnlichkeeits-Sortierfunktion *topMatches* unverändert eingesetzt werden.

__Aufgabe:__

1. Implementieren Sie eine Funktion, welche das Bewertungsdictionary *critics* derart transformiert, dass die Funktionen `sim_euclid`, `sim_pearson` und `topMatches` für das Item-basierte CF unverändert eingesetzt werden können. Die transformierte Matrix soll unter dem Namen *transCritics* abgespeichert werden.
2. Schreiben Sie eine Funktion `calculateSimilarItems`, die aus der transformierten Matrix *transCritics* ein Dictionary berechnet, welches die Ähnlichkeit zwischen allen Filmen beschreibt. Die Keys des Dictionary sind die Filmnamen. Die Values sind geordnete Listen, welche die Funktion `topMatches` zurückgibt, wenn sie für die Filme (nicht für die User) aufgerufen wird. Dieses Dictionary wird an das aufrufende Programm zurück geben. 
3. Schreiben Sie eine Funktion `getRecommendedItems`, welche basierend auf dem im unten aufgeführten Beispiel dargestellten Verfahren unter Vorgabe der Bewertungsmatrix und der zu verwendenden Ähnlichkeitsfunktion Produktempfehlungen berechnet.
4. Testen Sie die Funktion indem Sie die Empfehlungen für Toby berechnen und mit den Werten in der unten aufgeführten Tabelle vergleichen

__Erläuterndes Beispiel:__

_Toby_ hat die Filme *The Night Listener*, *Lady in the Water* und *Just My Luck* noch nicht gekauft. Für diese Filme soll für *Toby* eine Empfehlung berechnet werden. Gekauft und bewertet hat *Toby* die Filme *Snakes on a plane*, *Superman Returns* und *You and me and Dupree*. Diese bereits vorhandenen Filme bilden die erste Spalte der unten dargestellten Matrix. In der zweiten Spalte befinden sich _Toby's_ Bewertungen dieser Filme. Die Spalten 3,5 und 7 enthalten die Ähnlichkeitswerte (mit *calculateSimilarItems* unter Verwendung des normierten euklidischen Ähnlichkeitsmaßes berechnet) zwischen den drei von *Toby* noch nicht gekauften Filmen und den drei von _Toby_ bewerteten Filmen. Diese Ähnlichkeitswerte werden jeweils mit _Toby's_ Bewertungen multipliziert. Das Resultat dieser Multiplikation befindet sich in den Spalten 4,6 und 8. Der finale Empfehlungswert für die von _Toby_ noch nicht gekauften Filme wird berechnet in dem in den Spalten 4,6 und 8 zunächst die Summe über die Werte dieser Spalte in den drei oberen Zeilen berechnet wird und durch die Summe über die Werte der Spalten 3,5 und 7 geteilt wird. Im Fall, dass die *Pearson-Korrelation* zwischen den Filmen als Ähnlichkeitswert herangezogen wird, können negative Ähnlichkeitswerte auftreten. Dann soll in die Berechnung eines Empfehlungswert für Film A nur dann die Bewertung von Film B einfließen, wenn der Korrelationswert zwischen beiden $>0$ ist.  


<img src="https://maucher.home.hdm-stuttgart.de/Pics/recommenderFilmItemBased.PNG" style="width:1000px" align="center"> -->


In [ ]:
def transform_dict(prefs):
    all_films = []
    all_persons = []
    
    for i in prefs.keys():
        all_persons.append(i)
    
    for i in range(0, len(all_persons)):
        films_of_user_i = prefs[all_persons[i]]
        for j in films_of_user_i:
            if(j not in all_films):
                all_films.append(j)

    transformed_dict = {}

    for i in range(0,len(all_films)):
        temp_dict = {}
        
        for pers in all_persons:
            if(all_films[i] in prefs[pers].keys()):
                temp_dict[pers] = prefs[pers][all_films[i]]
            
        transformed_dict[all_films[i]] = temp_dict

    return transformed_dict

In [23]:
transCritics = transform_dict(critics)

In [24]:
transCritics

{'Lady in the Water': {'Lisa Rose': 2.5,
  'Gene Seymour': 3.0,
  'Michael Phillips': 2.5,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 3.0},
 'Snakes on a Plane': {'Lisa Rose': 3.5,
  'Gene Seymour': 3.5,
  'Michael Phillips': 3.0,
  'Claudia Puig': 3.5,
  'Mick LaSalle': 4.0,
  'Jack Matthews': 4.0,
  'Toby': 4.5},
 'Just My Luck': {'Lisa Rose': 3.0,
  'Gene Seymour': 1.5,
  'Claudia Puig': 3.0,
  'Mick LaSalle': 2.0},
 'Superman Returns': {'Lisa Rose': 3.5,
  'Gene Seymour': 5.0,
  'Michael Phillips': 3.5,
  'Claudia Puig': 4.0,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 5.0,
  'Toby': 4.0},
 'You, Me and Dupree': {'Lisa Rose': 2.5,
  'Gene Seymour': 3.5,
  'Claudia Puig': 2.5,
  'Mick LaSalle': 2.0,
  'Jack Matthews': 3.5,
  'Toby': 1.0},
 'The Night Listener': {'Lisa Rose': 3.0,
  'Gene Seymour': 3.0,
  'Michael Phillips': 4.0,
  'Claudia Puig': 4.5,
  'Mick LaSalle': 3.0,
  'Jack Matthews': 3.0}}

In [25]:
def calculateSimilarItems(similarity):
    result_dict = {}
    films = transCritics.keys()
    for film in films:
        result_dict[film] = top_matches(transCritics, film, similarity)
    return result_dict

In [26]:
calculateSimilarItems('sim_pearson')

{'Lady in the Water': [['Snakes on a Plane', np.float64(0.7637626158259734)],
  ['Superman Returns', np.float64(0.48795003647426655)],
  ['You, Me and Dupree', np.float64(0.3333333333333333)],
  ['The Night Listener', np.float64(-0.6123724356957947)],
  ['Just My Luck', np.float64(-0.9449111825230683)]],
 'Snakes on a Plane': [['Lady in the Water', np.float64(0.7637626158259734)],
  ['Superman Returns', np.float64(0.11180339887498948)],
  ['Just My Luck', np.float64(-0.3333333333333333)],
  ['The Night Listener', np.float64(-0.5663521139548541)],
  ['You, Me and Dupree', np.float64(-0.6454972243679027)]],
 'Just My Luck': [['The Night Listener', np.float64(0.5555555555555556)],
  ['Snakes on a Plane', np.float64(-0.3333333333333333)],
  ['Superman Returns', np.float64(-0.422890031611031)],
  ['You, Me and Dupree', np.float64(-0.48566186425718266)],
  ['Lady in the Water', np.float64(-0.9449111825230683)]],
 'Superman Returns': [['You, Me and Dupree', np.float64(0.657951694959769)],
  [

In [47]:
def getRecommendedItems(prefs, person, similarity):
    #Get all items
    all_films = []
    for i in prefs.keys():
        all_films.append(i)
   
    #Get items used/rated by person
    films_watched_by_person = []
    for i in all_films:
        if(person in prefs[i]):
            films_watched_by_person.append(i)

    #Get items not used/rated by person
    films_not_watched_by_person = []
    for i in all_films:
        if(person not in prefs[i]):
            films_not_watched_by_person.append(i)

    #Get the person's ratings for the Items
    film_rating_of_person = {}
    for i in films_watched_by_person:
        film_rating_of_person[i] = prefs[i][person]

    #Get similarity scores of the items
    ratings_dict = calculateSimilarItems(similarity)
    
    #Create a dictionary with all relevant item similarity scores
    film_sim_scores = {}
    for i in films_not_watched_by_person:
        temp_dict = {}
        for j in ratings_dict[i]:
            if(j[0] in films_watched_by_person):
                if(j[1] < 0):
                    temp_dict[j[0]] = 0
                elif(j[1] >= 0):
                    temp_dict[j[0]] = j[1]
                
        film_sim_scores[i] = temp_dict     
        
    rating_times_sim = {}
    
    #Calculate item sim score reletaive to person sim score
    for i in films_not_watched_by_person:
        temp_dict = {}
        for j in film_rating_of_person:
            sim_score = film_sim_scores[i][j]
            product = film_rating_of_person[j] * sim_score
            temp_dict[j] = product
        rating_times_sim[i] = temp_dict

    movie_recommendations_items = {}

    #Calculate recommendation scores for item
    for i in films_not_watched_by_person:
        sum_sim_scores = 0
        sum_product_scores = 0
        for j in films_watched_by_person:
            sum_sim_scores = sum_sim_scores + film_sim_scores[i][j]
            sum_product_scores = sum_product_scores + rating_times_sim[i][j]
        if(sum_sim_scores <= 0):
            print(f"Der Film: {i} ist nicht empfehlendwert!") # for pearson, if the sum of the sim scores is smaller or equal 0 (item not relevant for recommandation)
        elif(sum_sim_scores > 0):
            movie_recommendations_items[i] = sum_product_scores/sum_sim_scores

    movie_recommendations_items_sorted = dict(sorted(movie_recommendations_items.items(), key=lambda item: item[1], reverse = True))
    return movie_recommendations_items_sorted

In [28]:
getRecommendedItems(transCritics, 'Toby', 'sim_euclid')

{'The Night Listener': np.float64(3.2044909601608804),
 'Lady in the Water': np.float64(3.082136961799338),
 'Just My Luck': np.float64(3.041861869079099)}

In [29]:
getRecommendedItems(transCritics, 'Toby', 'sim_pearson')

Der Film: Just My Luck ist nicht empfehlendwert!
Der Film: The Night Listener ist nicht empfehlendwert!


{'Lady in the Water': np.float64(3.610031066802179)}

## last.fm Musikempfehlungen

__Aufgabe:__

1. Installieren Sie das Package pylast. Stellen Sie durch Aufruf der Funktion *network=pylast.LastFMNetwork()* eine Verbindung zu *last.fm* her. Beim Aufruf der Funktion wird die Anmeldung und Authentifizierung durchgeführt. Die Funktion gibt ein Objekt der Klasse *Network* zurück. Über dieses Objekt können Methoden, wie

    * *get_artist("kuenstlerName")* (liefert Objekt der Klasse *Artist*)
    * *get_album("albumName")* (liefert Objekt der Klasse *Album*)
    * *get_track("songName")* (liefert Objekt der Klasse *Track*)
    * *get_user("userName"):* (liefert Objekt der Klasse *Tag*)
    * usw.
    
      aufgerufen werden. Die Menge aller verfügbaren Klassen und deren Attribute und Methoden können dem Modul _pylast.py_ entnommen werden. In der untenstehenden Codel Zelle können Sie beispielsweise über das *Network*-Objekt die Methode `get_artist("BandIhrerWahl")` aufrufen um die von lastFM berechneten ähnlichsten Bands anzuzeigen.

2. Führen Sie die untenstehende Code Zelle aus um eine vordefinierte Gruppe von Benutzern, passend zu Band 'Slipknot' zu bekommen. Falls Sie ein lastfm Konto haben können Sie manuell auch andere Bands und Benutzer eintragen.
3. Implementieren Sie eine Funktion `createLastfmUserDict()`. Dieser Funktion soll, die oben angelegte Liste von *User*-Objekten _group_ übergeben werden. Für jeden User in *group* sollen die 20 beliebtesten Bands mit der Methode `topartists=*User*.get_top_artists()[0:20]` bestimmt werden. Die Methode gibt eine Liste von *Artist*-Objekt/Gewichtung-Paaren zurück. Die Gewichtungen von Objekten werden in diesem Versuch nicht benötigt. Auf das *i.te* *Artist*-Objekt selbst kann mit `topartists[i].item` zugegriffen werden. Die Menge aller Bands, die auf diese Weise gesammelt werden, wird im folgenden mit _AllBands_ bezeichnet. D.h. in *AllBands* befinden sich alle Bands, die für mindestens einen User in *group* zu den Top-20 gehören. Nun soll ein verschachteltes Dictionary mit Namen *userDict* wie folgt angelegt werden:

    * Die Keys sind die Namen der _User_-Objekte in _group_. Auf den Namen eines Objekts kann mit `get_name()` zugegriffen werden.
    * Die Values sind selbst wieder Dictionaries, deren Keys die Namen der Bands in *AllBands* sind. Achten Sie auch hier darauf, dass Sie nicht das *Artist*-Objekt selbst, sondern dessen Namen als Key verwenden. 
    * Für den User *a* und die Band *b* ist der Value `userDict[a][b]= 1`, falls *b* zu den Top-20 des Users *a* gehört. Andernfalls ist `userDict[a][b]= 0`. 
    
    Das derart angelegte Dictionary soll von der Funktion zurückgegeben werden. 
4. Wählen Sie jetzt einen beliebigen User aus *group*. Bestimmen Sie zu diesem User die ähnlichsten User in *group* durch Anwendung der im ersten Teilversuch implementierten Funktion `topMatches()`. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik angewandt werden.
5. Bestimmen Sie dann für den gewählten User Band-Empfehlungen durch Anwendung der im ersten Teilversuch implementierten Funktion `getRecommendations()`. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik, danach die Russel_Rao Metrik, angewandt werden.     
6. Diskutieren Sie das Ergebnis

In [30]:
nw=pylast.LastFMNetwork(api_key="993a5bd9d79a98a53677570368d55acd", api_secret="9b8de0b57903ac007cdd8ec9003b341e", username="pythonlab")
band='Slipknot'
art1 = nw.get_artist(band)
print("Most similar (as calculated by lastFM) for artist: ", band)

for it in art1.get_similar(5):
    print("%3.3f \t %s"%(it.match, it.item))

print("\nApply predefined group of users")
usernames=['BrunoJoS','DPREBOYE','MPistol40','NemoNightfall','SkyRif','Wags1382','Znapsen','cortapsyco','emill_67','sattuviitana']
group=[]
for u in usernames:
    u1 = nw.get_user(u)
    group.append(u1)
print(group)

Most similar (as calculated by lastFM) for artist:  Slipknot
1.000 	 Korn
0.710 	 Stone Sour
0.695 	 System of a Down
0.571 	 Mudvayne
0.547 	 Vended

Apply predefined group of users
[pylast.User('BrunoJoS', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('DPREBOYE', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('MPistol40', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('NemoNightfall', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('SkyRif', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b341e', '', 'pythonlab', '')), pylast.User('Wags1382', pylast.LastFMNetwork('993a5bd9d79a98a53677570368d55acd', '9b8de0b57903ac007cdd8ec9003b34

In [31]:
def createLastfmUserDict(group):
    AllBands = []

    for user in group:
        topartists = user.get_top_artists()[0:20]
        for artist in topartists:
            AllBands.append(artist.item.name)

    userDict = {user.get_name():{band:0 for band in AllBands} for user in group}

    for user, user_obj in zip(userDict, group):
        topartists = user_obj.get_top_artists()[:20]
        for artist in topartists:
            if artist.item.name in userDict[user].keys():
                userDict[user][artist.item.name] = 1

    return userDict  

In [32]:
user_dict = createLastfmUserDict(group)

In [33]:
pprint(user_dict)

{'BrunoJoS': {'30 Seconds to Mars': 1,
              '36 Crazyfists': 0,
              'A Perfect Circle': 0,
              'AFI': 0,
              'Alexisonfire': 0,
              'All Time Low': 0,
              'Amon Amarth': 0,
              'Amorphis': 0,
              'Arch Enemy': 0,
              'Arctic Monkeys': 1,
              'Atoms for Peace': 0,
              'Avenged Sevenfold': 0,
              'Bayside': 0,
              'Beach House': 0,
              'Becoming the Archetype': 0,
              'Ben Howard': 0,
              'Billy Talent': 0,
              'Black Label Society': 0,
              'Black Veil Brides': 0,
              'Boston': 0,
              'Breaking Benjamin': 0,
              'Bring Me the Horizon': 0,
              'Bullet for My Valentine': 0,
              'Calvin Harris': 0,
              'Cat Power': 0,
              'Children of Bodom': 0,
              'Circa Survive': 0,
              'Coheed and Cambria': 0,
              'Cradle of Filt

In [34]:
top_matches(prefs=user_dict, person='DPREBOYE', similarity='sim_euclid')

[['BrunoJoS', np.float64(0.9615384615384615)],
 ['MPistol40', np.float64(0.9615384615384615)],
 ['SkyRif', np.float64(0.9615384615384615)],
 ['emill_67', np.float64(0.9615384615384615)],
 ['NemoNightfall', np.float64(0.9605261285528738)],
 ['Wags1382', np.float64(0.9605261285528738)],
 ['Znapsen', np.float64(0.9605261285528738)],
 ['cortapsyco', np.float64(0.9605261285528738)],
 ['sattuviitana', np.float64(0.9605261285528738)]]

In [48]:
getRecommendations(prefs=user_dict, person='DPREBOYE', similarity='sim_euclid')

[[np.float64(0.6667056830543407), 'System of a Down'],
 [np.float64(0.5556465937934617), 'Korn'],
 [np.float64(0.44435340620653824), 'Five Finger Death Punch'],
 [np.float64(0.3335284152717037), 'Red Hot Chili Peppers'],
 [np.float64(0.3334113661086815), 'Linkin Park'],
 [np.float64(0.3334113661086815), 'Papa Roach'],
 [np.float64(0.3334113661086815), 'Stone Sour'],
 [np.float64(0.33329431694565925), 'Rammstein'],
 [np.float64(0.33329431694565925), 'Metallica'],
 [np.float64(0.22235227684780248), 'Muse'],
 [np.float64(0.22235227684780248), 'Foo Fighters'],
 [np.float64(0.2222352276847802), '30 Seconds to Mars'],
 [np.float64(0.2222352276847802), 'Three Days Grace'],
 [np.float64(0.2222352276847802), 'In Flames'],
 [np.float64(0.2222352276847802), 'Limp Bizkit'],
 [np.float64(0.2222352276847802), 'Pendulum'],
 [np.float64(0.2222352276847802), 'Lamb of God'],
 [np.float64(0.2222352276847802), 'Sabaton'],
 [np.float64(0.2222352276847802), 'Pink Floyd'],
 [np.float64(0.222118178521758), 'T

In [49]:
getRecommendations(prefs=user_dict, person='DPREBOYE', similarity='sim_RusselRao')

[[0.6923076923076923, 'System of a Down'],
 [0.6153846153846154, 'Korn'],
 [0.4615384615384615, 'Red Hot Chili Peppers'],
 [0.3846153846153846, 'Linkin Park'],
 [0.3846153846153846, 'Papa Roach'],
 [0.3846153846153846, 'Stone Sour'],
 [0.3846153846153846, 'Five Finger Death Punch'],
 [0.3076923076923077, 'Muse'],
 [0.3076923076923077, 'Rammstein'],
 [0.3076923076923077, 'Foo Fighters'],
 [0.3076923076923077, 'Metallica'],
 [0.23076923076923075, '30 Seconds to Mars'],
 [0.23076923076923075, 'Three Days Grace'],
 [0.23076923076923075, 'In Flames'],
 [0.23076923076923075, 'Limp Bizkit'],
 [0.23076923076923075, 'Pendulum'],
 [0.23076923076923075, 'Lamb of God'],
 [0.23076923076923075, 'Sabaton'],
 [0.23076923076923075, 'Pink Floyd'],
 [0.15384615384615385, 'Queens of the Stone Age'],
 [0.15384615384615385, 'Radiohead'],
 [0.15384615384615385, 'Sum 41'],
 [0.15384615384615385, 'Green Day'],
 [0.15384615384615385, 'The Killers'],
 [0.15384615384615385, 'Royal Blood'],
 [0.15384615384615385, 

**Antwort**:

* Die Euklidische Werte und die Russel-Rao Werte sind in diesem Fall nicht weit von einander entfernt
* Die Russel-Rao Ähnlichkeit wird vor allem bei einem binären Format verwendet, was hier der Fall ist
* Bei einem binären Format ergeben sich bei der Euklidischen Ähnlichkeit ähnliche Werte wie bei der Russel-Rao Ähnlichkeit
* Die Pearson Ähnlichkeit ergibt schlechte Werte, da nur 20 von 150 (Bei diesen Benutzern) Werten betrachtet werden, wodurch die Ähnlichkeitswerte sehr gering sind

## Hypothese

### Die Normierung der euklidischen Ähnlichekit hat Einfluss auf die Recommendtions, da der, durch die unterschieldlichen Längen der Vektoren, verursachte Fehler, Einfluss auf die Ähnlichkeitswerte der Person hat.

### Toby - Top Matches

In [50]:
top_matches(person='Toby', prefs=critics, similarity='sim_euclid', norm=True)

[['Mick LaSalle', np.float64(0.6666666666666666)],
 ['Claudia Puig', np.float64(0.6246387977050463)],
 ['Lisa Rose', np.float64(0.615911621788925)],
 ['Michael Phillips', np.float64(0.5584815598877471)],
 ['Jack Matthews', np.float64(0.5227744249483389)],
 ['Gene Seymour', np.float64(0.5108747069239427)]]

In [51]:
top_matches(person='Toby', prefs=critics, similarity='sim_euclid', norm=False)

[['Mick LaSalle', np.float64(0.4)],
 ['Michael Phillips', np.float64(0.38742588672279304)],
 ['Claudia Puig', np.float64(0.3567891723253309)],
 ['Lisa Rose', np.float64(0.3483314773547883)],
 ['Jack Matthews', np.float64(0.2674788903885893)],
 ['Gene Seymour', np.float64(0.25824569976124334)]]

### Toby - Recommendations

In [52]:
getRecommendations(prefs=critics, person='Toby', similarity='sim_euclid', norm=True)

[[np.float64(3.4273481378103883), 'The Night Listener'],
 [np.float64(2.795737031164006), 'Lady in the Water'],
 [np.float64(2.407392750287351), 'Just My Luck']]

In [53]:
getRecommendations(prefs=critics, person='Toby', similarity='sim_euclid', norm=False)

[[np.float64(3.457128694491423), 'The Night Listener'],
 [np.float64(2.778584003814924), 'Lady in the Water'],
 [np.float64(2.4224820423619167), 'Just My Luck']]

### Fazit - Toby

* Die Ähnlichkeiten verschlechtern sich deutlich
* Bei der Reihenfolge der Top-Matches Ausgabe ändern sich die ähnlichsten Benutzer, jedoch nicht das Top Match
* Dies hat einen minimalen Einfluss auf die Recommendations-Werte für den Benutzer Toby, jedoch nicht für den Top Recommendation für den Benutzer Toby

### Michael Phillips - Top Matches

In [54]:
top_matches(person='Michael Phillips', prefs=critics, similarity='sim_euclid', norm=True)

[['Lisa Rose', np.float64(0.7815501047457912)],
 ['Claudia Puig', np.float64(0.7759907622602041)],
 ['Mick LaSalle', np.float64(0.7166996059009808)],
 ['Gene Seymour', np.float64(0.6737986373538911)],
 ['Jack Matthews', np.float64(0.6534537935444722)],
 ['Toby', np.float64(0.5584815598877471)]]

In [55]:
top_matches(person='Michael Phillips', prefs=critics, similarity='sim_euclid', norm=False)

[['Claudia Puig', np.float64(0.5358983848622454)],
 ['Lisa Rose', np.float64(0.4721359549995794)],
 ['Mick LaSalle', np.float64(0.38742588672279304)],
 ['Toby', np.float64(0.38742588672279304)],
 ['Gene Seymour', np.float64(0.3405424265831667)],
 ['Jack Matthews', np.float64(0.32037724101704074)]]

### Michael Phillips - Recommendations

In [56]:
getRecommendations(prefs=critics, person='Michael Phillips', similarity='sim_euclid', norm=True)

[[np.float64(2.5315339166769295), 'You, Me and Dupree'],
 [np.float64(2.4140520198936697), 'Just My Luck']]

In [57]:
getRecommendations(prefs=critics, person='Michael Phillips', similarity='sim_euclid', norm=False)

[[np.float64(2.482581708640552), 'Just My Luck'],
 [np.float64(2.453379230569188), 'You, Me and Dupree']]

### Fazit - Michael Phillips

* Bei Top Matches verändern sich die Werte genau wie bei Toby und werden niedriger. Hier jedoch ändern sich auch das Top Match für Michael Phillips
* Bei den Recommendations hat es ebenfalls einen minimalen Einfluss auf die Recommendations-Werte und es verändern sich die Top Recommenadtion für den Benutzer Michael Phillips

### last.fm - Top Matches

In [58]:
top_matches(prefs=user_dict, person='DPREBOYE', similarity='sim_euclid', norm=True)

[['BrunoJoS', np.float64(0.9615384615384615)],
 ['MPistol40', np.float64(0.9615384615384615)],
 ['SkyRif', np.float64(0.9615384615384615)],
 ['emill_67', np.float64(0.9615384615384615)],
 ['NemoNightfall', np.float64(0.9605261285528738)],
 ['Wags1382', np.float64(0.9605261285528738)],
 ['Znapsen', np.float64(0.9605261285528738)],
 ['cortapsyco', np.float64(0.9605261285528738)],
 ['sattuviitana', np.float64(0.9605261285528738)]]

In [59]:
top_matches(prefs=user_dict, person='DPREBOYE', similarity='sim_euclid', norm=False)

[['BrunoJoS', np.float64(0.14285714285714285)],
 ['MPistol40', np.float64(0.14285714285714285)],
 ['SkyRif', np.float64(0.14285714285714285)],
 ['emill_67', np.float64(0.14285714285714285)],
 ['NemoNightfall', np.float64(0.13957875683699938)],
 ['Wags1382', np.float64(0.13957875683699938)],
 ['Znapsen', np.float64(0.13957875683699938)],
 ['cortapsyco', np.float64(0.13957875683699938)],
 ['sattuviitana', np.float64(0.13957875683699938)]]

### last.fm - Fazit

* Die Ähnlichkeitswerte verändern sich deutlich und werden niedriger, aber...
* die Reihenfolge der Top Matches ändert sich nicht

## Fazit

* Bei den Vektoren der Film-Empfehlungen haben wir einen Unterschied verursacht durch die unterschiedliche Länge der Vektoren.
* Bei den Vektoren der last.fm-Empfehlungen ergibt sich kein Unterschied, da die Vektoren durch die 0, 1 Codierung immer die gleiche Länge $\sqrt{20}$ haben. Aber auch der relative Abstand vor und nach der Normierung ist gleich, da bei der Normierng immer durch die Länge der Künstler geteilt wird.
* Bei den Filmbewertungen wird durch eine Zahl von 1-6 normiert (Abhängig von der Zahl der bewerteten Filme), somit bleibt der relative Abstand vor und nach der Normierung gleich.

<img src="hypothese_img.png" width="700px">

* Links sind nicht normiert und normiert, Vektor 1 und Vektor 2 am ähnlichsten.
* Rechts sind bei nicht normiert haben Vektor 2 und Vektor 3 am nächsten an einander und sind somit am ähnlichsten. Wenn man die normiert sind jedoch Vektor 1 und Vektor 3 am ähnlichsten.
* Links (last.fm) wird mit einer gleicher Länge normiert
* Rechts (Filmbewertungen) unterschiedlich lange Vektoren, unterschiedliche Anzahl der Bewertungen pro Benutzer und somit unterschiedliche Normierung

Die Hypotehse trifft zu, hängt aber stark davon ab wie groß der Längenunterschied der Vektoren ist.